In [7]:
# Start by making a connection
import pyodbc 
import pandas as pd

server = 'talentdataplayground.database.windows.net' 
database = 'ChewZilla_ecommerce' 
username = 'candidate_andri'
password = 'agkn898gkrfgj7iws*ee7&t'
driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()



In [8]:
# Check if all the tables exist 
# and what they look like

sql1 = """
    select top 10 *
    from Products;
"""

sql2 = """
    select top 10 *
    from OrderItems;
"""

sql3 = """
    select top 10 *
    from Orders;
"""

print(pd.read_sql(sql1,cnxn))
print(pd.read_sql(sql2,cnxn))
print(pd.read_sql(sql3,cnxn))

   ProductId            ProductName               Category
0        1.0         Cyborg Glasses    Time Travel Devices
1        2.0    Chewbacca Telescope      Cosmic Telescopes
2        3.0  Captain America Chips        Dinosaur Treats
3        4.0       Wonder Woman Kit   Jungle Survival Kits
4        5.0      Boba Fett Candies        Dinosaur Treats
5        6.0   Captain Marvel Troop  Enchanted Forest Maps
6        7.0     Wonder Woman World          Treasure Maps
7        8.0       Darth Vader Nose    Cybernetic Implants
8        9.0              Loki Ship      Space Exploration
9       10.0     Green Lantern Tube      Cosmic Telescopes
   OrderItemId  OrderId  ProductId  Quantity  Price
0          1.0      1.0       51.0        10    190
1          2.0      1.0       56.0         8     96
2          3.0      1.0       13.0         4     28
3          4.0      1.0       48.0         7     49
4          5.0      2.0       19.0        10    140
5          6.0      3.0       50.0     

C:\Users\andri\AppData\Local\Temp\ipykernel_12084\4167602370.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(sql1,cnxn))
C:\Users\andri\AppData\Local\Temp\ipykernel_12084\4167602370.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(sql2,cnxn))
C:\Users\andri\AppData\Local\Temp\ipykernel_12084\4167602370.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(sql3,cnxn))


**Task 1**

a. Identify ChewZilla's top 5 customers by total order amount for 2023, who all seem to have a
predilection for spaceship parts?

b. Find the top 3 products in the 'Dinosaur Treats' category that have sold the most units in 2023.

In [9]:
#Start by looking for dublicates
sql = """
    select OrderId, count(*) as No
    from Orders
    group by OrderId
    having count(*) > 1
"""
print(pd.read_sql(sql,cnxn))

sql = """
    select OrderItemId, count(*) as No
    from OrderItems
    group by OrderItemId  
    having count(*) > 1
"""
print(pd.read_sql(sql,cnxn))

sql = """
    select ProductId, count(*) as No
    from Products
    group by ProductId
    having count(*) > 1
"""
print(pd.read_sql(sql,cnxn))

# These need to be removed

   OrderId  No
0     19.0   2
1     61.0   2
2     83.0   2
   OrderItemId  No
0         23.0   2
1         24.0   2
2         25.0   2
3         26.0   2
4         27.0   2
5        405.0   2
6        406.0   2
7        407.0   2
8        408.0   2
Empty DataFrame
Columns: [ProductId, No]
Index: []


C:\Users\andri\AppData\Local\Temp\ipykernel_12084\1253584533.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(sql,cnxn))
C:\Users\andri\AppData\Local\Temp\ipykernel_12084\1253584533.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(sql,cnxn))
C:\Users\andri\AppData\Local\Temp\ipykernel_12084\1253584533.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(sql,cnxn))


In [10]:
# A top 5 customers by total for 2023
# who all seem to have a predilection for spaceship parts - I take that to mean just an interesting comment, not filter by Category
pd.read_sql(""" 
    with temp_distinct as (
        select distinct OrderId, CustomerId, TotalAmount 
        from Orders
        where year(OrderDate) = 2023
    ),
    sum_orders as (
        select  
            td.CustomerId, 
            sum(td.TotalAmount) as TotalAmount
        from temp_distinct td
        group by td.CustomerId
    ),
    sum_orders2 as 
    (
    select
        so.CustomerId,
        so.TotalAmount,
        row_number() over(order by so.TotalAmount desc) as rn
    from sum_orders so
    )
    select * 
    from sum_orders2 so2
    where so2.rn <= 5
    """, cnxn)


C:\Users\andri\AppData\Local\Temp\ipykernel_12084\1786606694.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,CustomerId,TotalAmount,rn
0,93.0,760,1
1,45.0,633,2
2,7.0,614,3
3,15.0,498,4
4,21.0,483,5


In [ ]:
# 2 Top 3 products in the 'Dinosaur Treats' category that have sold the most units in 2023.

In [11]:
pd.read_sql(""" 
    with temp as (
        select distinct o.OrderId, oi.OrderItemId, p.ProductId, oi.Quantity 
        from OrderItems oi
        inner join Products p on p.ProductId = oi.ProductId
            inner join Orders o on o.OrderId = oi.OrderId
        where year(o.OrderDate) = 2023
            and p.Category = 'Dinosaur Treats'
    ),
    ranking as (
        select ProductId, sum(Quantity) as TotalItemsSold, row_number() over(order by sum(Quantity) desc) as rank
        from temp
        group by ProductId
    )
    select * 
    from ranking
    where rank <= 3
    """, cnxn)

C:\Users\andri\AppData\Local\Temp\ipykernel_12084\1068398897.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,ProductId,TotalItemsSold,rank
0,45.0,39,1
1,41.0,27,2
2,3.0,18,3


c. Calculate the month-over-month growth in total sales for the year 2023. Month-over-month
growth is a good indicator of sales trend and is calculated as ((Sales this month - Sales last month) /
Sales last month) * 100.

In [12]:
pd.read_sql(""" 
    with temp as 
    (    
        select distinct OrderDate, TotalAmount
        from Orders
    ),
    temp2 as
    (
        select 
            eomonth(OrderDate) as OrderMonth,
            sum(TotalAmount) TotalAmount
        from temp
        group by eomonth(OrderDate)
    ),
    create_lag as 
    (
        select 
            *,
            lag(totalAmount, 1)  over (order by OrderMonth) TotalAmountLM
        from temp2
    )
    select * ,
    ((TotalAmount*1.0 - TotalAmountLM) / TotalAmountLM) *100 as MoM_Growth --*1.0 to convert to float
    from create_lag 
    where year(orderMonth) = 2023
    """, cnxn)

C:\Users\andri\AppData\Local\Temp\ipykernel_12084\3598963550.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,OrderMonth,TotalAmount,TotalAmountLM,MoM_Growth
0,2023-01-31,10289,9982,3.075536
1,2023-02-28,1648,10289,-83.982894


d. Identify the product that has seen the largest increase in sales quantity from one order to the
next in 2023. This could help ChewZilla identify products that suddenly became popular.

In [13]:
# I'm assuming I can go by OrderId 

pd.read_sql(""" 
    with distinct_temp as 
    (
        select distinct o.OrderId, oi.OrderItemId, p.ProductId, p.ProductName, p.Category, oi.Quantity, o.OrderDate
        from OrderItems oi
        inner join Products p on p.ProductId = oi.ProductId
            inner join Orders o on o.OrderId = oi.OrderId
    ),
    totals as
    (
        select 
            ProductId, 
            OrderId,
            OrderDate,
            ProductName, 
            Category, 
            sum(Quantity) as TotalQuantity
        from distinct_temp
        group by ProductId, OrderId, OrderDate, ProductName, Category
    ),
    lagging as
    (
        select 
            ProductId,
            OrderId,
            ProductName, 
            Category, 
            TotalQuantity,
            OrderDate,
            lag(TotalQuantity,1) over(partition by ProductId order by OrderId) as TotalQuantityLM,
            ((TotalQuantity*1.0 - lag(TotalQuantity,1) over(partition by ProductId order by OrderId)) / lag(TotalQuantity,1) over(partition by ProductId order by OrderId)) *100 as Growth
        from totals
    ),
    ranking as
    (
        select 
            OrderId,
            ProductName,
            Category, 
            Growth as GrowthBetweenOrders,
            row_number() over( order by Growth desc) as rn 
        from lagging
        where Growth is not null
            and year(OrderDate) = 2023
     )
     select * from ranking
     where rn <= 10
     order by rn 
     
    """, cnxn)

C:\Users\andri\AppData\Local\Temp\ipykernel_12084\3831615718.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,OrderId,ProductName,Category,GrowthBetweenOrders,rn
0,187.0,Aquaman Jewel,Enchanted Jewelry,900.0,1
1,141.0,Beauty scape,Exotic Fruits,900.0,2
2,189.0,Chewbacca Shock,Ninja Weapons,900.0,3
3,158.0,The Flash Scientist,Jungle Survival Kits,700.0,4
4,196.0,Wonder Woman World,Treasure Maps,700.0,5
5,147.0,Cyborg Knife,Time Travel Devices,700.0,6
6,201.0,Cyclops Space Trap,Spaceship Parts,600.0,7
7,144.0,Captain Marvel Gauntlet,Spaceship Parts,500.0,8
8,202.0,Jean Grey Jacket,Robot Accessories,400.0,9
9,204.0,C-3PO Hyper Solid,Space Helmets,350.0,10


**Task 2: Error Detection and Query Optimization**


ChewZilla's intern, a well-intentioned dinosaur named T-Rexie, has written an SQL query to find out
the total quantity of each product sold. Her query is as follows:


SELECT p.ProductId, p.ProductName, SUM(oi.Quantity)

FROM Products p, OrderItems oi

WHERE p.ProductId = oi.ProductId 

GROUP BY oi.ProductId

ORDER BY SUM(oi.Quantity) DESC


T-Rexie may have short arms, but she has a big heart for SQL. Could you check her query for errors
and inefficiencies and provide an optimized version?

Reccomendations: 
- Group by ProductId and ProductName from the Product table 
- Inner join the Product table instead of just selecting everything
- Start by dealing with dublicates


In [14]:
pd.read_sql(""" 
SELECT distinct 
    p.ProductId, 
    p.ProductName, 
    SUM(oi.Quantity) as TotalQuantity
FROM Products p
inner join OrderItems oi on oi.ProductId = p.ProductId
WHERE p.ProductId = oi.ProductId 
GROUP BY p.ProductId, p.ProductName
ORDER BY SUM(oi.Quantity) DESC
""", cnxn)

C:\Users\andri\AppData\Local\Temp\ipykernel_12084\2999197772.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,ProductId,ProductName,TotalQuantity
0,51.0,Vision 3D,107
1,53.0,Barbie Grapes,100
2,4.0,Wonder Woman Kit,92
3,20.0,Luke Skywalker Finger,92
4,22.0,Wonerland Berry,91
5,11.0,C-3PO Hyper Solid,81
6,25.0,Green Lantern Chair,80
7,10.0,Green Lantern Tube,76
8,5.0,Boba Fett Candies,76
9,33.0,Captain Marvel Gauntlet,76


Example Optional questions:

c. T-Rexie is also curious about the average quantity of each product sold per order. Could you
help her write an optimized query for this?

In [15]:
import qgrid
from ipywidgets import widgets
from IPython.display import display

query = pd.read_sql(""" 
with totals as
(
    SELECT distinct 
        p.ProductId, 
        p.ProductName, 
        oi.OrderId,
        SUM(oi.Quantity) as TotalQuantity
    FROM Products p
    inner join OrderItems oi on oi.ProductId = p.ProductId
    WHERE p.ProductId = oi.ProductId 
    GROUP BY p.ProductId, p.ProductName, oi.OrderId
    --ORDER BY SUM(oi.Quantity) DESC
)
select
    OrderId,
    ProductName,
    avg(TotalQuantity) as AverageQuantity
from totals
group by OrderId, ProductName
""", cnxn)

print(query)

     OrderId     ProductName  AverageQuantity
0       13.0  Ant-Man System                9
1       35.0  Ant-Man System                8
2       67.0  Ant-Man System               10
3       87.0  Ant-Man System                7
4      104.0  Ant-Man System                3
..       ...             ...              ...
600     71.0      Yoda Scarf                2
601     81.0      Yoda Scarf                3
602    117.0      Yoda Scarf                2
603    145.0      Yoda Scarf                5
604    169.0      Yoda Scarf                7

[605 rows x 3 columns]


C:\Users\andri\AppData\Local\Temp\ipykernel_12084\3847861473.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = pd.read_sql("""


**Task 3: Data Modeling and Architecture**

    
ChewZilla wants to launch a loyalty program, 'ChewZilla Galactic Rewards'. Customers earn points
for each order, redeemable for discounts on future purchases. The points vary by product category
and the amount spent.

a. Propose changes to the existing schema or new tables to support this exciting loyalty program.

b. Very briefly describe your design choices and the considerations made for data integrity and
performance.

**Product table**

Add a column for how many points each product is worth (column)


**Orders table**

Add a column for how many points each order was worth. We would get this number by multiplying by quantity ordered.


**Two new history tables for the loyalty points**

Since the points are redeamable we would need to know how many points customers have at any given day.
We would have to create a new table that accrues points per day. This table could include:

- StatusDate
- CustomerID
- ProductID

Knowing total points per product might be interesting to be able to analyze the data for marketing purposes.

Since it doesn't matter how the points were obtained (which product was bought) when it comes to spending the points, I would create a seperate table that holds total points per day (including spent points).
- StatusDate
- CustomerID
- TotalAccruedPoints
- TotalSpentPoints

We would add a new StatusDates for each table daily.

**Task 4: ETL Design**
    
In a surprising turn of events, ChewZilla has acquired "MartianMunch", another company
specializing in extraterrestrial snacks. ChewZilla now needs to integrate MartianMunch's sales data
with theirs. The snag is that MartianMunch's data is stored in a different database with a different
schema.

a. Briefly outline the steps you would take to design an ETL process for this scenario.

b. Highlight some potential issues that might arise and how you would address them.

There are generally three steps when designing an ETL process:
- Extraction
- Transformation
- Loading/analysis 

Seperating the steps makes the code more legible and easier to manage.

Secondly, we would need to choose what kind of tools to use to manage the ETL's. At my previous job the data engineer maintained the ETL processes with Microsoft SQL Server Integration Services (SSIS). At my current job my team manages the ETL's with SAS.

One point would be to try to avoid moving unnecessary data over so that the processing time won't be too slow.
Another point is to make sure the column types will match the ones in our current database and make transformations accordingly.

Adding primary keys and foreign keys will be helpful. We need to design this in a way so it fits our current database structure.

The data might be structured differently in the other database and it the structure might need to be either flattened or the opposite.

It would be helpful to keep the star schema model in mind when making design choices.

We would have to make sure to account for the other firm having some of the same customers and products and make sure that will not cause dublicates.
